# Wikipedia document finder for dumps

How does it work:
 what do you need:
 - a .csv with the links to wikipedia pages (column name for the links is 'url')
 - Parsed dataset (see joão's github, --> Downloading and extracting Wikidump for a specific language)


In [1]:
import wikipediaapi # pip3 install wikipedia-api
import pandas as pd
import numpy as np
import re
import bs4
import sys
import requests
import os

# read .csv with wiki links (needs URL column)
file_path = 'Topic spreadsheet docs - Blad1.csv'
documents_df = pd.read_csv(file_path)
doc_links = documents_df['url']

# filter for wiki links
doc_links = doc_links[doc_links.str.contains('nl.wikipedia')].tolist() # change nl. for different languages

# remove dupicates 
doc_links = set(doc_links)

# check for number of documents
print('Number of documents: {}'.format(len(doc_links)))

Number of documents: 337


In [2]:
# empty dict for info
data = {'title_wiki':[], 'title_system':[], 'web_link':[]}

# connect to wiki API, change nl for different languages
wiki_wiki = wikipediaapi.Wikipedia(
        language='nl', 
        extract_format=wikipediaapi.ExtractFormat.WIKI)


# wiki text extractor
def write_txt(web_link):
    """
    Finds the titles used in wikipedia based on the links
    
    web_link: str with a link to a wikipedia page
    """
    
    # get HTML File
    res = requests.get(web_link)
    res.raise_for_status()
    wiki = bs4.BeautifulSoup(res.text,'html.parser')
    
    # get title
    title_wiki = wiki.find(id='firstHeading').string
    title_system = re.subn("[-|*|\\s+|%]", '_', title_wiki)

    # save items to dict
    data['title_wiki'].append(title_wiki)
    data['title_system'].append(title_system[0])
    data['web_link'].append(web_link)
            
    
    return data

In [3]:
%%time
# sub = list(doc_links)[0:15] # remove this line when running for the whole dataset

# Getting requierments 
for i in doc_links:
    write_txt(i)

# Create data frame with titles
title_link = pd.DataFrame(data, columns=['title_wiki', 'title_system', 'web_link'])

title_link.head()
title_link.to_csv('nl_documents.csv',columns=['title_system','web_link'], index=False)

CPU times: user 22.8 s, sys: 637 ms, total: 23.4 s
Wall time: 1min 12s


In [7]:
%%time

# set rootdir to the location of the documents in the acx env.
rootdir = '/Users/jesher/Desktop/Master data science UvA/Semester 2/Thesis/acx_102/data/FullWikipedia'


# func to get individule documents for the wikipedia data
def wiki_doc_finder(abs_path, title_df):
    """
    
    """
    print("This is going to take a while, so grab a coffee!")
    
    doc_list = []
    
    # creating the file directories
    for subdir, dirs, files in os.walk(abs_path):
        for file in files:
            abs_path = os.path.join(subdir, file)
            
            with open(abs_path, 'r') as f:
                contents = f.read()

            # loop over all files in the df and wikipedia titles
            for items in contents.split('</doc>'):
                for title_wiki in title_df['title_wiki']:

                    # recreating the titles
                    wiki_title = 'title=' + '"' + title_wiki + '"'
                    title_system = re.subn("[-|*|\\s+|%]", '_', title_wiki)
                    
                    # matching the titles
                    if re.search(wiki_title, items):
                        doc_list.append(str(title_system[0]))
                        print(doc_list)

#                         # saving the documents 
#                         with open(('./Documents/' + str(title_system[0]) + '.txt'), 'w') as f:   
#                             f.write(items)
                            
    # removing excess files
    del contents, wiki_title, title_system
    
    return print("number of documents:{}".format(len(doc_list))), doc_list
    

# run function                
wiki_doc_finder(rootdir, title_link)

This is going to take a while, so grab a coffee!
['Titelbescherming']
['Titelbescherming', 'Pieter_Jacobus_Wemelsfelder']


KeyboardInterrupt: 

In [8]:
docs = wiki_doc_finder(rootdir, title_link)

This is going to take a while, so grab a coffee!
['Titelbescherming']
['Titelbescherming', 'Pieter_Jacobus_Wemelsfelder']
['Titelbescherming', 'Pieter_Jacobus_Wemelsfelder', 'Middle_East_respiratory_syndrome']
['Titelbescherming', 'Pieter_Jacobus_Wemelsfelder', 'Middle_East_respiratory_syndrome', 'Blockchain']
['Titelbescherming', 'Pieter_Jacobus_Wemelsfelder', 'Middle_East_respiratory_syndrome', 'Blockchain', 'Nationaal_park_Tijuca']
['Titelbescherming', 'Pieter_Jacobus_Wemelsfelder', 'Middle_East_respiratory_syndrome', 'Blockchain', 'Nationaal_park_Tijuca', 'Deviezen']
['Titelbescherming', 'Pieter_Jacobus_Wemelsfelder', 'Middle_East_respiratory_syndrome', 'Blockchain', 'Nationaal_park_Tijuca', 'Deviezen', 'Hollandsche_Beton_Maatschappij']
['Titelbescherming', 'Pieter_Jacobus_Wemelsfelder', 'Middle_East_respiratory_syndrome', 'Blockchain', 'Nationaal_park_Tijuca', 'Deviezen', 'Hollandsche_Beton_Maatschappij', 'Sigrid_Kaag']
['Titelbescherming', 'Pieter_Jacobus_Wemelsfelder', 'Middle_E

['Titelbescherming', 'Pieter_Jacobus_Wemelsfelder', 'Middle_East_respiratory_syndrome', 'Blockchain', 'Nationaal_park_Tijuca', 'Deviezen', 'Hollandsche_Beton_Maatschappij', 'Sigrid_Kaag', 'E.T._the_Extra_Terrestrial', 'Anne_Vondeling', 'Aandacht', 'Warmte_krachtkoppeling', 'Spoorlijn_Boxtel___Wesel', 'Centraal_station', 'Klassieke_conditionering', 'Bitumen', 'West_Berlijn', 'VU_medisch_centrum', 'Executeur', 'Gamma_uil', 'MIMO', 'Koninklijke_Belgische_Voetbalbond', 'Steenwijkerwold', 'American_Broadcasting_Company', 'Signaal_ruisverhouding', 'TACAN', 'VHF_omnidirectional_range_station', 'Aanbesteding', 'Tippelprostitutie']
['Titelbescherming', 'Pieter_Jacobus_Wemelsfelder', 'Middle_East_respiratory_syndrome', 'Blockchain', 'Nationaal_park_Tijuca', 'Deviezen', 'Hollandsche_Beton_Maatschappij', 'Sigrid_Kaag', 'E.T._the_Extra_Terrestrial', 'Anne_Vondeling', 'Aandacht', 'Warmte_krachtkoppeling', 'Spoorlijn_Boxtel___Wesel', 'Centraal_station', 'Klassieke_conditionering', 'Bitumen', 'West_Be

['Titelbescherming', 'Pieter_Jacobus_Wemelsfelder', 'Middle_East_respiratory_syndrome', 'Blockchain', 'Nationaal_park_Tijuca', 'Deviezen', 'Hollandsche_Beton_Maatschappij', 'Sigrid_Kaag', 'E.T._the_Extra_Terrestrial', 'Anne_Vondeling', 'Aandacht', 'Warmte_krachtkoppeling', 'Spoorlijn_Boxtel___Wesel', 'Centraal_station', 'Klassieke_conditionering', 'Bitumen', 'West_Berlijn', 'VU_medisch_centrum', 'Executeur', 'Gamma_uil', 'MIMO', 'Koninklijke_Belgische_Voetbalbond', 'Steenwijkerwold', 'American_Broadcasting_Company', 'Signaal_ruisverhouding', 'TACAN', 'VHF_omnidirectional_range_station', 'Aanbesteding', 'Tippelprostitutie', 'Luchtverkeersleider', 'Omzet', 'Springstaarten', 'Nederlandsche_Handel_Maatschappij', 'Amsterdamsche_Bank', 'LG_Group', 'Hogeschool_van_Amsterdam', 'Library_of_Congress', 'Langpootmuggen', 'BMW_Motorrad', 'Scouts_en_Gidsen_Vlaanderen', 'Ria_Beckers']
['Titelbescherming', 'Pieter_Jacobus_Wemelsfelder', 'Middle_East_respiratory_syndrome', 'Blockchain', 'Nationaal_park

['Titelbescherming', 'Pieter_Jacobus_Wemelsfelder', 'Middle_East_respiratory_syndrome', 'Blockchain', 'Nationaal_park_Tijuca', 'Deviezen', 'Hollandsche_Beton_Maatschappij', 'Sigrid_Kaag', 'E.T._the_Extra_Terrestrial', 'Anne_Vondeling', 'Aandacht', 'Warmte_krachtkoppeling', 'Spoorlijn_Boxtel___Wesel', 'Centraal_station', 'Klassieke_conditionering', 'Bitumen', 'West_Berlijn', 'VU_medisch_centrum', 'Executeur', 'Gamma_uil', 'MIMO', 'Koninklijke_Belgische_Voetbalbond', 'Steenwijkerwold', 'American_Broadcasting_Company', 'Signaal_ruisverhouding', 'TACAN', 'VHF_omnidirectional_range_station', 'Aanbesteding', 'Tippelprostitutie', 'Luchtverkeersleider', 'Omzet', 'Springstaarten', 'Nederlandsche_Handel_Maatschappij', 'Amsterdamsche_Bank', 'LG_Group', 'Hogeschool_van_Amsterdam', 'Library_of_Congress', 'Langpootmuggen', 'BMW_Motorrad', 'Scouts_en_Gidsen_Vlaanderen', 'Ria_Beckers', 'Suikerfabriek', 'The_Good,_the_Bad_and_the_Ugly', 'Caraïben', 'Mountainbike', 'Fast_moving_consumer_goods', 'Westfal

['Titelbescherming', 'Pieter_Jacobus_Wemelsfelder', 'Middle_East_respiratory_syndrome', 'Blockchain', 'Nationaal_park_Tijuca', 'Deviezen', 'Hollandsche_Beton_Maatschappij', 'Sigrid_Kaag', 'E.T._the_Extra_Terrestrial', 'Anne_Vondeling', 'Aandacht', 'Warmte_krachtkoppeling', 'Spoorlijn_Boxtel___Wesel', 'Centraal_station', 'Klassieke_conditionering', 'Bitumen', 'West_Berlijn', 'VU_medisch_centrum', 'Executeur', 'Gamma_uil', 'MIMO', 'Koninklijke_Belgische_Voetbalbond', 'Steenwijkerwold', 'American_Broadcasting_Company', 'Signaal_ruisverhouding', 'TACAN', 'VHF_omnidirectional_range_station', 'Aanbesteding', 'Tippelprostitutie', 'Luchtverkeersleider', 'Omzet', 'Springstaarten', 'Nederlandsche_Handel_Maatschappij', 'Amsterdamsche_Bank', 'LG_Group', 'Hogeschool_van_Amsterdam', 'Library_of_Congress', 'Langpootmuggen', 'BMW_Motorrad', 'Scouts_en_Gidsen_Vlaanderen', 'Ria_Beckers', 'Suikerfabriek', 'The_Good,_the_Bad_and_the_Ugly', 'Caraïben', 'Mountainbike', 'Fast_moving_consumer_goods', 'Westfal

...

['Titelbescherming', 'Pieter_Jacobus_Wemelsfelder', 'Middle_East_respiratory_syndrome', 'Blockchain', 'Nationaal_park_Tijuca', 'Deviezen', 'Hollandsche_Beton_Maatschappij', 'Sigrid_Kaag', 'E.T._the_Extra_Terrestrial', 'Anne_Vondeling', 'Aandacht', 'Warmte_krachtkoppeling', 'Spoorlijn_Boxtel___Wesel', 'Centraal_station', 'Klassieke_conditionering', 'Bitumen', 'West_Berlijn', 'VU_medisch_centrum', 'Executeur', 'Gamma_uil', 'MIMO', 'Koninklijke_Belgische_Voetbalbond', 'Steenwijkerwold', 'American_Broadcasting_Company', 'Signaal_ruisverhouding', 'TACAN', 'VHF_omnidirectional_range_station', 'Aanbesteding', 'Tippelprostitutie', 'Luchtverkeersleider', 'Omzet', 'Springstaarten', 'Nederlandsche_Handel_Maatschappij', 'Amsterdamsche_Bank', 'LG_Group', 'Hogeschool_van_Amsterdam', 'Library_of_Congress', 'Langpootmuggen', 'BMW_Motorrad', 'Scouts_en_Gidsen_Vlaanderen', 'Ria_Beckers', 'Suikerfabriek', 'The_Good,_the_Bad_and_the_Ugly', 'Caraïben', 'Mountainbike', 'Fast_moving_consumer_goods', 'Westfal

['Titelbescherming', 'Pieter_Jacobus_Wemelsfelder', 'Middle_East_respiratory_syndrome', 'Blockchain', 'Nationaal_park_Tijuca', 'Deviezen', 'Hollandsche_Beton_Maatschappij', 'Sigrid_Kaag', 'E.T._the_Extra_Terrestrial', 'Anne_Vondeling', 'Aandacht', 'Warmte_krachtkoppeling', 'Spoorlijn_Boxtel___Wesel', 'Centraal_station', 'Klassieke_conditionering', 'Bitumen', 'West_Berlijn', 'VU_medisch_centrum', 'Executeur', 'Gamma_uil', 'MIMO', 'Koninklijke_Belgische_Voetbalbond', 'Steenwijkerwold', 'American_Broadcasting_Company', 'Signaal_ruisverhouding', 'TACAN', 'VHF_omnidirectional_range_station', 'Aanbesteding', 'Tippelprostitutie', 'Luchtverkeersleider', 'Omzet', 'Springstaarten', 'Nederlandsche_Handel_Maatschappij', 'Amsterdamsche_Bank', 'LG_Group', 'Hogeschool_van_Amsterdam', 'Library_of_Congress', 'Langpootmuggen', 'BMW_Motorrad', 'Scouts_en_Gidsen_Vlaanderen', 'Ria_Beckers', 'Suikerfabriek', 'The_Good,_the_Bad_and_the_Ugly', 'Caraïben', 'Mountainbike', 'Fast_moving_consumer_goods', 'Westfal

['Titelbescherming', 'Pieter_Jacobus_Wemelsfelder', 'Middle_East_respiratory_syndrome', 'Blockchain', 'Nationaal_park_Tijuca', 'Deviezen', 'Hollandsche_Beton_Maatschappij', 'Sigrid_Kaag', 'E.T._the_Extra_Terrestrial', 'Anne_Vondeling', 'Aandacht', 'Warmte_krachtkoppeling', 'Spoorlijn_Boxtel___Wesel', 'Centraal_station', 'Klassieke_conditionering', 'Bitumen', 'West_Berlijn', 'VU_medisch_centrum', 'Executeur', 'Gamma_uil', 'MIMO', 'Koninklijke_Belgische_Voetbalbond', 'Steenwijkerwold', 'American_Broadcasting_Company', 'Signaal_ruisverhouding', 'TACAN', 'VHF_omnidirectional_range_station', 'Aanbesteding', 'Tippelprostitutie', 'Luchtverkeersleider', 'Omzet', 'Springstaarten', 'Nederlandsche_Handel_Maatschappij', 'Amsterdamsche_Bank', 'LG_Group', 'Hogeschool_van_Amsterdam', 'Library_of_Congress', 'Langpootmuggen', 'BMW_Motorrad', 'Scouts_en_Gidsen_Vlaanderen', 'Ria_Beckers', 'Suikerfabriek', 'The_Good,_the_Bad_and_the_Ugly', 'Caraïben', 'Mountainbike', 'Fast_moving_consumer_goods', 'Westfal

['Titelbescherming', 'Pieter_Jacobus_Wemelsfelder', 'Middle_East_respiratory_syndrome', 'Blockchain', 'Nationaal_park_Tijuca', 'Deviezen', 'Hollandsche_Beton_Maatschappij', 'Sigrid_Kaag', 'E.T._the_Extra_Terrestrial', 'Anne_Vondeling', 'Aandacht', 'Warmte_krachtkoppeling', 'Spoorlijn_Boxtel___Wesel', 'Centraal_station', 'Klassieke_conditionering', 'Bitumen', 'West_Berlijn', 'VU_medisch_centrum', 'Executeur', 'Gamma_uil', 'MIMO', 'Koninklijke_Belgische_Voetbalbond', 'Steenwijkerwold', 'American_Broadcasting_Company', 'Signaal_ruisverhouding', 'TACAN', 'VHF_omnidirectional_range_station', 'Aanbesteding', 'Tippelprostitutie', 'Luchtverkeersleider', 'Omzet', 'Springstaarten', 'Nederlandsche_Handel_Maatschappij', 'Amsterdamsche_Bank', 'LG_Group', 'Hogeschool_van_Amsterdam', 'Library_of_Congress', 'Langpootmuggen', 'BMW_Motorrad', 'Scouts_en_Gidsen_Vlaanderen', 'Ria_Beckers', 'Suikerfabriek', 'The_Good,_the_Bad_and_the_Ugly', 'Caraïben', 'Mountainbike', 'Fast_moving_consumer_goods', 'Westfal

number of documents:300


In [10]:
len(docs)

2

In [15]:
docs = docs[1]

In [29]:
title_link['in_docs'] = title_link['title_system'].isin(docs)

In [30]:
test = title_link[title_link['in_docs'] == True]

In [26]:
title_link

,title_wiki,title_system,web_link,in_docs
0,Soapserie,Soapserie,https://nl.wikipedia.org/wiki/Soapserie,True
1,Theorie,Theorie,https://nl.wikipedia.org/wiki/Theorie,True
2,Verenigd Koninkrijk,Verenigd_Koninkrijk,https://nl.wikipedia.org/wiki/Verenigd_Koninkrijk,True
3,Bitumen,Bitumen,https://nl.wikipedia.org/wiki/Bitumen,True
4,Waterschapsverkiezingen,Waterschapsverkiezingen,https://nl.wikipedia.org/wiki/Waterschapsverki...,True
...,...,...,...,...
332,Nederlandse Omroep Stichting,Nederlandse_Omroep_Stichting,https://nl.wikipedia.org/wiki/Nederlandse_Omro...,True
333,Tefra,Tefra,https://nl.wikipedia.org/wiki/Tefra,True
334,Sony,Sony,https://nl.wikipedia.org/wiki/Sony,True
335,Luchtvaartmaatschappij,Luchtvaartmaatschappij,https://nl.wikipedia.org/wiki/Luchtvaartmaatsc...,True


In [31]:
test

,title_wiki,title_system,web_link,in_docs
0,Soapserie,Soapserie,https://nl.wikipedia.org/wiki/Soapserie,True
1,Theorie,Theorie,https://nl.wikipedia.org/wiki/Theorie,True
2,Verenigd Koninkrijk,Verenigd_Koninkrijk,https://nl.wikipedia.org/wiki/Verenigd_Koninkrijk,True
3,Bitumen,Bitumen,https://nl.wikipedia.org/wiki/Bitumen,True
4,Waterschapsverkiezingen,Waterschapsverkiezingen,https://nl.wikipedia.org/wiki/Waterschapsverki...,True
...,...,...,...,...
332,Nederlandse Omroep Stichting,Nederlandse_Omroep_Stichting,https://nl.wikipedia.org/wiki/Nederlandse_Omro...,True
333,Tefra,Tefra,https://nl.wikipedia.org/wiki/Tefra,True
334,Sony,Sony,https://nl.wikipedia.org/wiki/Sony,True
335,Luchtvaartmaatschappij,Luchtvaartmaatschappij,https://nl.wikipedia.org/wiki/Luchtvaartmaatsc...,True


In [32]:
test.to_csv('nl_documents_final.csv',columns=['title_system','web_link'], index=False)